# EMI AND LOAN CALCULATION

## Calculate emi taking average of all major banks across 20 years
### average rates :

     | Bank / Lender         | Typical Floating Rate (p.a.)|
     | --------------------- | ------------------ |
     | SBI                   | ~7.50% – 8.70%+ |
     | HDFC Bank             | ~7.90% – 13.20% |
     | ICICI Bank            | ~8.75% – 9.80% |
     | Kotak Mahindra        | ~7.99% – 13.3% |
     | Union Bank of India   | ~7.45% – 10% |
     | Canara Bank           | ~7.40% – 10.25% |
     | Central Bank of India | ~7.35% – 9.40% |
     | Bajaj Housing Finance | ~7.40% – 10.25% |
     | Indian Bank           | ~7.40% – 8.80% |

we take the average rate as : 
# assumed_home_loan_rate = 8.0


### For FY 2025-26 (assessment year 2026-27), the new tax regime (optional under section 115BAC) has lower and simplified tax rates:

     ₹0 – ₹4 L: Nil

     ₹4 L – ₹8 L: 5%

     ₹8 L – ₹12 L: 10%

     ₹12 L – ₹16 L: 15%

     ₹16 L – ₹20 L: 20%

     ₹20 L – ₹24 L: 25%

     Above ₹24 L: 30%

### Under the old tax regime (still allowed if you choose it when filing ITR):

     Section 24(b): Interest deduction

          Up to ₹2,00,000 per year for self-occupied property

          No limit for rented out properties

     Section 80(C): Principal repayment

          Deduction up to ₹1,50,000 per year


# we are gonna be using both the tax regimes and at last we choose the one which is beneficial (as it is still allowed if you choose it when filing ITR )


# what this code does right now:
     How much loan a buyer can realistically take
     Whether a property is affordable given income constraints
     The true EMI burden and its composition (interest vs principal)  
     The risk profile of the purchase before comparing it with renting



## Data Inputs Used

### Property-Level Inputs (from spreadsheet)

     Property price (listing price)

### User / Investor Inputs

     Gross annual income

     Home loan tenure (20 years)

     Bank interest rates (floating, from multiple lenders)

### Regulatory & Market Assumptions

     RBI loan-to-value (LTV) norms

     Conservative EMI-to-income cap

     Floating-rate home loans


firstly we calculate emi ratio i.e emi as a perrcentage of net monthly income


## Before any loan calculation, the model: 
     Computes tax liability under old and new tax regimes Automatically selects the lower tax regime Derives net annual income

## Maximum Loan Eligibility (Income-Constrained)

     Using the EMI ceiling, interest rate, and tenure:

     The model reverse-calculates the maximum loan allowed by income

## Down Payment Modeling (RBI-Compliant)

     Instead of assuming a fixed down payment, the model uses price-based minimum down payment rules aligned with RBI LTV norms:

     | Property Price | Minimum Down Payment |
     | -------------- | -------------------- |
     | ≤ ₹30 lakh     | 10% |
     | ₹30–75 lakh    | 20% |
     | > ₹75 lakh     | 25% |


## Actual Loan Determination (Final Constraint)

     The final loan amount is determined as:

     Minimum of:

     Loan required after down payment

     Maximum loan allowed by income

## EMI Calculation and Amortization

     For the finalized loan:

          Monthly EMI is calculated

          Full amortization schedule is generated

     EMI is split into:

          Interest component

          Principal component

     Additionally:

          Monthly data is aggregated into year-wise EMI splits

## EMI Stress Ratio

EMI as a percentage of net monthly income

Loan Cap Classification

     PROPERTY_CAPPED: Loan limited by property price

     INCOME_CAPPED: Loan limited by income affordability

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv("kolkata.csv")
print(df.columns.tolist())

['Name', 'Address', 'Bedrooms', 'Price', 'Rent', 'Area', 'Furnishing']


In [ ]:

# ============================================================
# 1. TAX CALCULATION (OLD & NEW REGIME)
# ============================================================

def tax_old_regime(taxable_income):
    slabs = [(250000, 0.0), (500000, 0.05), (1000000, 0.20), (float("inf"), 0.30)]
    tax, prev = 0, 0
    for limit, rate in slabs:
        if taxable_income > prev:
            tax += (min(taxable_income, limit) - prev) * rate
            prev = limit
    return 0 if taxable_income <= 500000 else tax


def tax_new_regime(taxable_income):
    slabs = [
        (400000, 0.0), (800000, 0.05), (1200000, 0.10),
        (1600000, 0.15), (2000000, 0.20),
        (2400000, 0.25), (float("inf"), 0.30)
    ]
    tax, prev = 0, 0
    for limit, rate in slabs:
        if taxable_income > prev:
            tax += (min(taxable_income, limit) - prev) * rate
            prev = limit
    return 0 if taxable_income <= 1275000 else tax

def emi_stress_ratio(monthly_emi, net_annual_income):

    """
    EMI as % of net monthly income
    """
    
    net_monthly_income = net_annual_income / 12
    return monthly_emi / net_monthly_income

def loan_cap_type(property_price, down_payment, loan_amount, tolerance=1e-6):
    
    """
    Determines what capped the loan:
    - PROPERTY_CAPPED
    - INCOME_CAPPED
    """

    required_loan = property_price - down_payment

    if abs(loan_amount - required_loan) < tolerance:
        return "PROPERTY_CAPPED"
    else:
        return "INCOME_CAPPED"
    

def risk_bucket(emi_ratio):
    """
    Industry-standard stress buckets
    """
    if emi_ratio <= 0.30:
        return "LOW_RISK"
    elif emi_ratio <= 0.40:
        return "MODERATE_RISK"
    elif emi_ratio <= 0.50:
        return "HIGH_RISK"
    else:
        return "VERY_HIGH_RISK"

def risk_warnings(cap_type, emi_ratio):
    warnings = []

    if cap_type == "INCOME_CAPPED":
        warnings.append("Loan capped by income — affordability constrained")

    if emi_ratio > 0.40:
        warnings.append("High EMI stress — limited cash-flow buffer")

    if emi_ratio > 0.50:
        warnings.append("Very high risk — vulnerable to rate hikes or income shocks")

    if not warnings:
        warnings.append("Comfortable affordability")

    return "; ".join(warnings)


def net_income_after_tax(gross_income):
    return gross_income - min(
        tax_old_regime(gross_income),
        tax_new_regime(gross_income)
    )

# ============================================================
# 2. HOME LOAN RATE (FLOATING POINT AVERAGE)
# ============================================================

def average_home_loan_rate(bank_rates_fp):
    return float(np.mean(bank_rates_fp))

# ============================================================
# 3. DOWN PAYMENT (PRICE-BASED, RBI LTV)
# ============================================================

def min_down_payment_pct(property_price):
    if property_price <= 3_000_000:
        return 0.10
    elif property_price <= 7_500_000:
        return 0.20
    else:
        return 0.25

# ============================================================
# 4. EMI & LOAN MATH
# ============================================================

def calculate_emi(loan_amount, annual_rate, tenure_years):
    r = annual_rate / 12 / 100
    n = tenure_years * 12
    return loan_amount * r * (1 + r)**n / ((1 + r)**n - 1)


def loan_from_emi(emi, annual_rate, tenure_years):
    r = annual_rate / 12 / 100
    n = tenure_years * 12
    return emi * ((1 + r)**n - 1) / (r * (1 + r)**n)


def emi_amortization(loan_amount, annual_rate, tenure_years):
    emi = calculate_emi(loan_amount, annual_rate, tenure_years)
    r = annual_rate / 12 / 100
    n = tenure_years * 12

    balance = loan_amount
    rows = []

    for m in range(1, n + 1):
        interest = balance * r
        principal = emi - interest
        balance -= principal

        rows.append({
            "month": m,
            "emi": emi,
            "interest": interest,
            "principal": principal,
            "balance": max(balance, 0)
        })

    return pd.DataFrame(rows)


def yearly_emi_split(amort_df):
    amort_df["year"] = (amort_df["month"] - 1) // 12 + 1
    return amort_df.groupby("year").agg({
        "emi": "sum",
        "interest": "sum",
        "principal": "sum"
    }).reset_index()

# ============================================================
# 5. PROPERTY-LEVEL CALCULATION
# ============================================================

def process_property_row(
    property_price,
    gross_annual_income,
    avg_rate,
    tenure_years,
    emi_ratio
):
    net_income = net_income_after_tax(gross_annual_income)

    max_emi = (net_income / 12) * emi_ratio
    max_loan = loan_from_emi(max_emi, avg_rate, tenure_years)

    dp_pct = min_down_payment_pct(property_price)
    down_payment = property_price * dp_pct

    required_loan = property_price - down_payment
    loan_amount = min(required_loan, max_loan)

    monthly_emi = calculate_emi(loan_amount, avg_rate, tenure_years)

    amort_df = emi_amortization(loan_amount, avg_rate, tenure_years)
    yearly_df = yearly_emi_split(amort_df)

    # --- RISK METRICS ---
    stress = emi_stress_ratio(monthly_emi, net_income)
    cap_type = loan_cap_type(property_price, down_payment, loan_amount)
    risk = risk_bucket(stress)
    warning_text = risk_warnings(cap_type, stress)

    return {
        "property_price": property_price,
        "down_payment_pct": dp_pct,
        "down_payment_amount": down_payment,
        "loan_amount": loan_amount,
        "monthly_emi": monthly_emi,
        "emi_to_income_ratio": stress,
        "loan_cap_type": cap_type,
        "risk_bucket": risk,
        "risk_warning": warning_text,
        "year1_interest": yearly_df.loc[0, "interest"],
        "year1_principal": yearly_df.loc[0, "principal"]
    }


# ============================================================
# 6. RUN FOR ENTIRE SPREADSHEET
# ============================================================

def run_for_spreadsheet(
    csv_path,
    gross_annual_income,
    bank_rates_fp,
    tenure_years=20,
    emi_ratio=0.40
):
    df = pd.read_csv(csv_path)

    PRICE_COL = "Price"
    avg_rate = average_home_loan_rate(bank_rates_fp)

    results = []

    for _, row in df.iterrows():
        results.append(
            process_property_row(
                property_price=row[PRICE_COL],
                gross_annual_income=gross_annual_income,
                avg_rate=avg_rate,
                tenure_years=tenure_years,
                emi_ratio=emi_ratio
            )
        )

    return pd.DataFrame(results)

# ============================================================
# 7. EXAMPLE EXECUTION
# ============================================================

if __name__ == "__main__":

    bank_rates_fp = [7.85, 8.0, 8.1, 7.9, 8.2]

    output = run_for_spreadsheet(
        csv_path="kolkata.csv",
        gross_annual_income=1800000,  # ₹18 LPA
        bank_rates_fp=bank_rates_fp
        )

    print(output.head())


   property_price  down_payment_pct  down_payment_amount   loan_amount  \
0       6630000.0              0.20            1326000.0  5.304000e+06   
1      27500000.0              0.25            6875000.0  6.530775e+06   
2      13400000.0              0.25            3350000.0  6.530775e+06   
3      15100000.0              0.25            3775000.0  6.530775e+06   
4       7490000.0              0.20            1498000.0  5.992000e+06   

    monthly_emi  emi_to_income_ratio    loan_cap_type    risk_bucket  \
0  44397.796686             0.324862  PROPERTY_CAPPED  MODERATE_RISK   
1  54666.666667             0.400000    INCOME_CAPPED  MODERATE_RISK   
2  54666.666667             0.400000    INCOME_CAPPED  MODERATE_RISK   
3  54666.666667             0.400000    INCOME_CAPPED  MODERATE_RISK   
4  50156.786905             0.367001  PROPERTY_CAPPED  MODERATE_RISK   

                                        risk_warning  year1_interest  \
0                          Comfortable affordabili